# Лабораторная работа №5
## Выполнил студент группы Самодуров Никита Сергеевич БПИ2303

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [23]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict
from collections import deque

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [24]:
# Алгоритм Кнута-Морриса-Пратта (КМП)
def kmp_search(text, pattern):
    n, m = len(text), len(pattern)
    
    # Префикс-функция
    pi = [0] * m
    j = 0
    for i in range(1, m):
        while j > 0 and pattern[i] != pattern[j]:
            j = pi[j - 1]
        if pattern[i] == pattern[j]:
            j += 1
            pi[i] = j
    
    # Поиск подстроки
    result = []
    j = 0
    for i in range(n):
        while j > 0 and text[i] != pattern[j]:
            j = pi[j - 1]
        if text[i] == pattern[j]:
            j += 1
        if j == m:
            result.append(i - m + 1)  # Найдено совпадение
            j = pi[j - 1]
    
    return result

# Функция для замера времени
def measure_time_kmp(text, pattern):
    start = t.now()
    result = kmp_search(text, pattern)
    end = t.now()
    return result, (end - start).total_seconds()


#### Упрощенный алгоритм Бойера-Мура

In [25]:

# Упрощенный алгоритм Бойера-Мура
def boyer_moore_search(text, pattern):
    n, m = len(text), len(pattern)
    if m == 0:
        return []

    # Таблица сдвигов
    shift = defaultdict(lambda: m)
    for i in range(m - 1):
        shift[pattern[i]] = m - 1 - i

    # Поиск
    result = []
    i = 0
    while i <= n - m:
        j = m - 1
        while j >= 0 and text[i + j] == pattern[j]:
            j -= 1
        if j < 0:
            result.append(i)
            i += shift[pattern[-1]]
        else:
            i += shift[text[i + m - 1]]
    
    return result

# Функция для замера времени
def measure_time_bm(text, pattern):
    start = t.now()
    result = boyer_moore_search(text, pattern)
    end = t.now()
    return result, (end - start).total_seconds()


### Запуск кода и вывод данных

In [26]:
# Ввод данных
text = "Пример строки для поиска"  
pattern = "поиска"  
case_sensitive = True  # True — учитывать регистр, False — игнорировать  

if not case_sensitive:
    text = text.lower()
    pattern = pattern.lower()

# Вызов алгоритмов и замер времени
res_kmp, time_kmp = measure_time_kmp(text, pattern)
res_bm, time_bm = measure_time_bm(text, pattern)
start = t.now()
res_builtin = [i for i in range(len(text)) if text.startswith(pattern, i)]
time_builtin = (t.now() - start).total_seconds()

# Вывод результатов
print("\nРезультаты поиска:")
print(f"КМП: {res_kmp}")
print(f"Бойера-Мура: {res_bm}")
print(f"str.find(): {res_builtin}")

print("\nВремя выполнения:")
print(f"КМП: {time_kmp:.6f} сек")
print(f"Бойера-Мура: {time_bm:.6f} сек")
print(f"str.find(): {time_builtin:.6f} сек")



Результаты поиска:
КМП: [18]
Бойера-Мура: [18]
str.find(): [18]

Время выполнения:
КМП: 0.000013 сек
Бойера-Мура: 0.000011 сек
str.find(): 0.000062 сек


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [27]:
from collections import deque

def is_solvable(puzzle):
    puzzle = [x for x in puzzle if x != 0]
    inversions = 0
    for i in range(len(puzzle)):
        for j in range(i + 1, len(puzzle)):
            if puzzle[i] > puzzle[j]:
                inversions += 1
    return inversions % 2 == 0

def get_neighbors(pos):
    x, y = pos
    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    return [(x + dx, y + dy) for dx, dy in moves if 0 <= x + dx < 4 and 0 <= y + dy < 4]

def solve_puzzle(puzzle):
    if not is_solvable(puzzle):
        return []

    goal = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
    puzzle_1d = puzzle[:]

    if puzzle_1d == goal:
        return []

    queue = deque([(puzzle_1d, [])])
    visited = set()

    while queue:
        state, path = queue.popleft()
        if state == goal:
            return path

        zero_pos = state.index(0)
        zero_x, zero_y = zero_pos // 4, zero_pos % 4

        for new_x, new_y in get_neighbors((zero_x, zero_y)):
            new_zero_pos = new_x * 4 + new_y
            new_state = state[:]
            new_state[zero_pos], new_state[new_zero_pos] = new_state[new_zero_pos], new_state[zero_pos]

            if str(new_state) not in visited:
                visited.add(str(new_state))
                queue.append((new_state, path + [state[new_zero_pos]]))

    return []

puzzle = [
    1, 2, 3, 4,
    5, 6, 7, 8,
    9, 11, 10, 12,
    13, 15, 14, 0
]

solution = solve_puzzle(puzzle)
if solution:
    print("Решение найдено!")
    print("Последовательность движений:", solution)
else:
    print("Решение невозможно")

Решение найдено!
Последовательность движений: [14, 10, 11, 15, 10, 14, 12, 11, 15, 10, 14, 15, 11, 12]


### Вывод